<a href="https://colab.research.google.com/github/thiagolermen/machine-learning-portifolio/blob/main/src/10-recurrent-neural-networks/PyTorch-gru.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Recurrent Neural Network - PyTorch

## Imports

In [14]:
import torch
import torchvision
import torch.nn as nn # All nn modules
import torch.optim as optim # Optimization algorithms
import torch.nn.functional as F # Functions as loss and activation
import torchvision.datasets as datasets # Standard datasets
import torchvision.transforms as transforms # Transformations for the dataset
from torch.utils.data import DataLoader # Easier dataset managment

from tqdm import tqdm  # For a nice progress bar

In [15]:
# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Model

In [16]:
# Hyperparameters
input_size = 28 #MNIST dataset
sequence_length = 28
num_layers = 2
hidden_size = 256
num_classes = 10
learning_rate = 0.003
batch_size = 64
num_epochs = 3

In [17]:
# Recurrent neural network (many-to-one)
class RNN_GRU(nn.Module):
  def __init__(self, input_size, hidden_size, num_layers, num_classes):
    super(RNN_GRU, self).__init__()
    self.hidden_size = hidden_size
    self.num_layers = num_layers
    self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
    self.fc = nn.Linear(hidden_size * sequence_length, num_classes)

  def forward(self, x):
    # Set initial hidden and cell states
    h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)

    # Forward propagate GRU
    out, _ = self.gru(x, h0)
    out = out.reshape(out.shape[0], -1)

    # Decode the hidden state of the last time step
    out = self.fc(out)
    return out

## Load data

In [18]:
train_dataset = datasets.MNIST(root="dataset/", train=True, transform=transforms.ToTensor(), download=True)
test_dataset = datasets.MNIST(root="dataset/", train=False, transform=transforms.ToTensor(), download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

## Train

In [19]:
# Initialize network (try out just using simple RNN, or GRU, and then compare with LSTM)
model = RNN_GRU(input_size, hidden_size, num_layers, num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Train Network
for epoch in range(num_epochs):
  for batch_idx, (data, targets) in enumerate(tqdm(train_loader)):
    # Get data to cuda if possible
    data = data.to(device=device).squeeze(1)
    targets = targets.to(device=device)

    # forward
    scores = model(data)
    loss = criterion(scores, targets)

    # backward
    optimizer.zero_grad()
    loss.backward()

    # gradient descent update step/adam step
    optimizer.step()

100%|██████████| 938/938 [00:08<00:00, 117.09it/s]


## Test

In [20]:
def check_accuracy(loader, model):
  num_correct = 0
  num_samples = 0

  # Set model to eval
  model.eval()

  with torch.no_grad():
    for x, y in loader:
      x = x.to(device=device).squeeze(1)
      y = y.to(device=device)

      scores = model(x)
      _, predictions = scores.max(1)
      num_correct += (predictions == y).sum()
      num_samples += predictions.size(0)

  # Toggle model back to train
  model.train()
  return num_correct / num_samples

print(f"Accuracy on training set: {check_accuracy(train_loader, model)*100:2f}")
print(f"Accuracy on test set: {check_accuracy(test_loader, model)*100:.2f}")

Accuracy on training set: 98.446671
Accuracy on test set: 98.11
